In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split as TTS,  GridSearchCV  
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB as NB


import nltk
from nltk.corpus import stopwords, sentiwordnet, wordnet
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

import spacy

from typing import List
from pprint import pprint


import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim

import tqdm

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping


import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jakob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import spacy
# spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

In [3]:
lotr_train = pd.read_csv('lotr_train.csv')
lotr_test = pd.read_csv('lotr_test.csv')
imp_char = ["FRODO", "SAM", "GANDALF", "PIPPIN", "MERRY", "GOLLUM", "GIMLI", "THEODEN", "FARAMIR", "SAURON", "ARAGORN", "SMEAGOL"]

def common_label_removal(data):
    mask = data["char"].isin(imp_char)
    data.loc[~ mask, "char"] = "Rest"
    mask2 = data['char'] == 'Rest'
    data = data[~mask2]
    return data

lotr_train = common_label_removal(lotr_train)
lotr_test = common_label_removal(lotr_test)

In [4]:
nlp = spacy.load("en_core_web_sm")

def find_max_length(data):
    return max(max(len(nlp(dialogue)) for dialogue in set) for set in data)

max_length = find_max_length([lotr_train['dialog'], lotr_test['dialog']])

def word2vec_df(data, max_length):
    # Extracting norm vector values
    word_vectors = []
    for dialogue in data:
        tokens = nlp(dialogue)
        dialogue_vectors = [token.vector_norm for token in tokens]
        word_vectors.append(dialogue_vectors)

    # Padding 
    for i in range(len(word_vectors)):
        word_vectors[i] += [0] * (max_length - len(word_vectors[i]))

    df = pd.DataFrame(word_vectors)
    df.columns = [f"word_{i}" for i in range(1, max_length + 1)]
    return df  

train_w2v = word2vec_df(lotr_train['dialog'], max_length)
test_w2v = word2vec_df(lotr_test['dialog'], max_length)

# train_w2v.to_csv('train_w2v.csv', index=False)
# test_w2v.to_csv('test_w2v.csv', index=False)

In [5]:
train_w2v['char'] = lotr_train['char'].reset_index(drop=True)
test_w2v['char'] = lotr_test['char'].reset_index(drop=True)

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [7]:
label_encoder = LabelEncoder()
train_w2v['char'] = label_encoder.fit_transform(train_w2v['char'])
test_w2v['char'] = label_encoder.transform(test_w2v['char'])

In [8]:
from keras import layers
from keras.layers import LSTM, Dense, Dropout, BatchNormalization

In [9]:
from keras import Sequential, layers, Input, callbacks

In [10]:
train_w2v

,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,word_10,...,word_162,word_163,word_164,word_165,word_166,word_167,word_168,word_169,word_170,char
0,7.936516,11.274338,12.535010,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,2
1,7.807209,7.082729,10.126812,9.273765,7.891686,8.183380,10.324145,7.371899,9.027041,7.104752,...,0,0,0,0,0,0,0,0,0,8
2,8.937764,7.750165,8.915595,8.662035,6.803154,6.460621,11.343762,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,7
3,10.943976,7.265589,10.792604,13.414934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,7
4,11.383319,8.229084,10.143223,7.550912,9.198028,7.539784,7.757065,8.074544,7.358045,7.983031,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,8.870296,9.705757,7.965269,6.652018,8.039692,8.533687,8.299333,11.232112,12.855322,0.000000,...,0,0,0,0,0,0,0,0,0,3
1148,8.342788,7.922681,8.421476,8.497047,9.581614,10.287869,9.037190,7.900794,11.145840,7.884370,...,0,0,0,0,0,0,0,0,0,7
1149,9.338175,12.174479,7.489946,10.374016,8.709492,8.717413,8.328237,9.383992,6.920962,7.234212,...,0,0,0,0,0,0,0,0,0,3
1150,9.798004,8.581070,9.354257,13.826755,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,8


In [11]:
tr_X = train_w2v.loc[:,:'word_170']
tr_y = train_w2v.loc[:,'char']
te_X = test_w2v.loc[:,:'word_170']
te_y = test_w2v.loc[:,'char']

In [12]:
model = keras.Sequential([
    layers.Dense(64,activation='relu', input_dim=170),
    layers.BatchNormalization(),
    layers.Dropout(rate=0.3),
    layers.Dense(128, activation='selu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(254, activation='softmax'),
    layers.Dense(1)
])
model.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy']
              )

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=30, # how many epochs to wait before stopping
    restore_best_weights=True,
)
model.fit(tr_X, tr_y, 
          validation_data= [te_X,te_y],
          epochs=200, batch_size=3, callbacks=early_stopping)


Epoch 1/200
384/384 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1163 - loss: 5.5368e-07 - val_accuracy: 0.1082 - val_loss: 5.8048e-07
Epoch 2/200
384/384 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1128 - loss: 5.5325e-07 - val_accuracy: 0.1082 - val_loss: 5.8048e-07
Epoch 3/200
384/384 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1097 - loss: 5.7176e-07 - val_accuracy: 0.1082 - val_loss: 5.8048e-07
Epoch 4/200
384/384 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1104 - loss: 5.6870e-07 - val_accuracy: 0.1082 - val_loss: 5.8048e-07
Epoch 5/200
384/384 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1276 - loss: 5.5326e-07 - val_accuracy: 0.1082 - val_loss: 5.8048e-07
Epoch 6/200
384/384 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1220 - loss: 5.7429e-07 - val_accuracy: 0.1082 - val_loss: 5.8048e-07
Epoch 7/200
384/384 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1264 - loss: 5.4863e-07 - val_accuracy: 0.1082 - val_loss: 5.8048e-07
Epoch 8/200
384/384 ━━━━━━━━━━━━━━━━━━━━ 

In [16]:
loss, accuracy = model.evaluate(tr_X, tr_y)

36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1226 - loss: 5.6045e-07


In [14]:
predictions = model.predict(tr_X)

36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step


In [15]:
model.predict_classes(tr_X)

AttributeError: 'Sequential' object has no attribute 'predict_classes'